### Residual Network
Welcome to the first assignment of this week! You'll be building a very deep convolutional network, using Residual Networks (ResNets). In theory, very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by He et al., allow you to train much deeper networks than were previously feasible.

In [2]:
# pip uninstall h5py

In [19]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
from test_utils import *
import public_tests

%matplotlib inline

### Exercise 1 - identity_block

In [20]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    
    F1, F2, F3 = filters
    X_shortcut = X
    cache = []
    
    X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding="valid", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2, kernel_size=f, strides=(1, 1), padding="same", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F3, kernel_size=1, strides=(1, 1), padding="valid", kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

In [21]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                    initializer=lambda seed=0:constant(value=1),
                   training=False)

print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71236 192.71236 192.71236  96.85619]
  [ 96.85619  96.85619  96.85619  48.9281 ]]

 [[578.1371  578.1371  578.1371  290.56854]
  [290.56854 290.56854 290.56854 146.78427]]]
96.85619


### 3.2 - The Convolutional Block

The ResNet "convolutional block" is the second block type. You can use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path.

In [22]:
def convolutional_block(X, f, filters, s=2, training=True, initializer=glorot_uniform):
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    # Main path
        # First component
    X = Conv2D(filters=F1, kernel_size=1, strides=(s,s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
        # Second component
    X = Conv2D(filters=F2, kernel_size=f, strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
        # Third component
    X = Conv2D(filters=F3, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    
    # Shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

In [23]:
# from outputs import convolutional_block_output1, convolutional_block_output2

np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis=0).astype(np.float32)

A = convolutional_block(X, f=2, filters=[2, 4, 6], training=False)
print(A[0])

tf.Tensor(
[[[1.3564789  0.         0.71636885 0.5440538  0.51367795 0.        ]
  [1.382489   0.         0.71729904 0.60515594 0.52714205 0.        ]]

 [[1.5120733  0.         0.64612526 0.74976003 0.47505397 0.        ]
  [1.6203711  0.         0.60756105 0.9147083  0.46440464 0.        ]]], shape=(2, 2, 6), dtype=float32)


### 4 - Building Your First ResNet Model (50 layers)


In [24]:
def ResNet50(input_shape=(64, 64, 3), classes=6):
    
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides=2, kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    
    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    
    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])
    
    # Output layer
    X = AveragePooling2D((2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer=glorot_uniform(seed=0))(X)
    
    # create model
    model = Model(inputs=X_input, outputs=X)
    
    return model

In [25]:
model = ResNet50(input_shape=(64, 64, 3), classes=6)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 70, 70, 3)   0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_67 (Conv2D)             (None, 32, 32, 64)   9472        ['zero_padding2d_1[0][0]']       
                                                                                                  
 batch_normalization_67 (BatchN  (None, 32, 32, 64)  256         ['conv2d_67[0][0]']        

                                                                                                  
 conv2d_77 (Conv2D)             (None, 15, 15, 256)  16640       ['activation_69[0][0]']          
                                                                                                  
 batch_normalization_77 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_77[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_22 (Add)                   (None, 15, 15, 256)  0           ['activation_67[0][0]',          
                                                                  'batch_normalization_77[0][0]'] 
                                                                                                  
 activation_70 (Activation)     (None, 15, 15, 256)  0           ['add_22[0][0]']                 
          

 add_25 (Add)                   (None, 8, 8, 512)    0           ['activation_76[0][0]',          
                                                                  'batch_normalization_87[0][0]'] 
                                                                                                  
 activation_79 (Activation)     (None, 8, 8, 512)    0           ['add_25[0][0]']                 
                                                                                                  
 conv2d_88 (Conv2D)             (None, 8, 8, 128)    65664       ['activation_79[0][0]']          
                                                                                                  
 batch_normalization_88 (BatchN  (None, 8, 8, 128)   512         ['conv2d_88[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

                                                                                                  
 batch_normalization_98 (BatchN  (None, 4, 4, 256)   1024        ['conv2d_98[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_89 (Activation)     (None, 4, 4, 256)    0           ['batch_normalization_98[0][0]'] 
                                                                                                  
 conv2d_99 (Conv2D)             (None, 4, 4, 256)    590080      ['activation_89[0][0]']          
                                                                                                  
 batch_normalization_99 (BatchN  (None, 4, 4, 256)   1024        ['conv2d_99[0][0]']              
 ormalization)                                                                                    
          

 activation_99 (Activation)     (None, 4, 4, 256)    0           ['batch_normalization_108[0][0]']
                                                                                                  
 conv2d_109 (Conv2D)            (None, 4, 4, 1024)   263168      ['activation_99[0][0]']          
                                                                                                  
 batch_normalization_109 (Batch  (None, 4, 4, 1024)  4096        ['conv2d_109[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_32 (Add)                   (None, 4, 4, 1024)   0           ['activation_97[0][0]',          
                                                                  'batch_normalization_109[0][0]']
                                                                                                  
 activatio

                                                                                                  
 add_35 (Add)                   (None, 2, 2, 2048)   0           ['activation_106[0][0]',         
                                                                  'batch_normalization_119[0][0]']
                                                                                                  
 activation_109 (Activation)    (None, 2, 2, 2048)   0           ['add_35[0][0]']                 
                                                                                                  
 average_pooling2d_1 (AveragePo  (None, 1, 1, 2048)  0           ['activation_109[0][0]']         
 oling2D)                                                                                         
                                                                                                  
 flatten_1 (Flatten)            (None, 2048)         0           ['average_pooling2d_1[0][0]']    
          

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [28]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T


model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10


2022-06-21 09:28:24.689942: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


34/34 [==============================] - 18s 461ms/step - loss: 1.6044 - accuracy: 0.4787
Epoch 2/10
34/34 [==============================] - 17s 485ms/step - loss: 0.4800 - accuracy: 0.8333
Epoch 3/10
34/34 [==============================] - 16s 471ms/step - loss: 0.4016 - accuracy: 0.8630
Epoch 4/10
34/34 [==============================] - 15s 454ms/step - loss: 0.3301 - accuracy: 0.8889
Epoch 5/10
34/34 [==============================] - 16s 463ms/step - loss: 0.3853 - accuracy: 0.8972
Epoch 6/10
34/34 [==============================] - 16s 467ms/step - loss: 0.2916 - accuracy: 0.9176
Epoch 7/10
34/34 [==============================] - 15s 449ms/step - loss: 0.1530 - accuracy: 0.9528
Epoch 8/10
34/34 [==============================] - 15s 449ms/step - loss: 0.1199 - accuracy: 0.9574
Epoch 9/10
34/34 [==============================] - 15s 450ms/step - loss: 0.0984 - accuracy: 0.9722
Epoch 10/10
34/34 [==============================] - 16s 456ms/step - loss: 0.0972 - accuracy: 0.9722


In [32]:
preds = model.evaluate(X_test, Y_test)
print("loss: " + str(preds[0]))
print("accuracy: " + str(preds[1]))

4/4 [==============================] - 0s 120ms/step - loss: 0.2671 - accuracy: 0.9167
loss: 0.2671208083629608
accuracy: 0.9166666865348816


In [35]:
model.save('resnet50.h5')

In [37]:
pre_trained_model = tf.keras.models.load_model('resnet50.h5')

In [38]:
pre_trained_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 70, 70, 3)   0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_67 (Conv2D)             (None, 32, 32, 64)   9472        ['zero_padding2d_1[0][0]']       
                                                                                                  
 batch_normalization_67 (BatchN  (None, 32, 32, 64)  256         ['conv2d_67[0][0]']        

                                                                                                  
 conv2d_77 (Conv2D)             (None, 15, 15, 256)  16640       ['activation_69[0][0]']          
                                                                                                  
 batch_normalization_77 (BatchN  (None, 15, 15, 256)  1024       ['conv2d_77[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_22 (Add)                   (None, 15, 15, 256)  0           ['activation_67[0][0]',          
                                                                  'batch_normalization_77[0][0]'] 
                                                                                                  
 activation_70 (Activation)     (None, 15, 15, 256)  0           ['add_22[0][0]']                 
          

 add_25 (Add)                   (None, 8, 8, 512)    0           ['activation_76[0][0]',          
                                                                  'batch_normalization_87[0][0]'] 
                                                                                                  
 activation_79 (Activation)     (None, 8, 8, 512)    0           ['add_25[0][0]']                 
                                                                                                  
 conv2d_88 (Conv2D)             (None, 8, 8, 128)    65664       ['activation_79[0][0]']          
                                                                                                  
 batch_normalization_88 (BatchN  (None, 8, 8, 128)   512         ['conv2d_88[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

                                                                                                  
 batch_normalization_98 (BatchN  (None, 4, 4, 256)   1024        ['conv2d_98[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_89 (Activation)     (None, 4, 4, 256)    0           ['batch_normalization_98[0][0]'] 
                                                                                                  
 conv2d_99 (Conv2D)             (None, 4, 4, 256)    590080      ['activation_89[0][0]']          
                                                                                                  
 batch_normalization_99 (BatchN  (None, 4, 4, 256)   1024        ['conv2d_99[0][0]']              
 ormalization)                                                                                    
          

 activation_99 (Activation)     (None, 4, 4, 256)    0           ['batch_normalization_108[0][0]']
                                                                                                  
 conv2d_109 (Conv2D)            (None, 4, 4, 1024)   263168      ['activation_99[0][0]']          
                                                                                                  
 batch_normalization_109 (Batch  (None, 4, 4, 1024)  4096        ['conv2d_109[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_32 (Add)                   (None, 4, 4, 1024)   0           ['activation_97[0][0]',          
                                                                  'batch_normalization_109[0][0]']
                                                                                                  
 activatio

                                                                                                  
 add_35 (Add)                   (None, 2, 2, 2048)   0           ['activation_106[0][0]',         
                                                                  'batch_normalization_119[0][0]']
                                                                                                  
 activation_109 (Activation)    (None, 2, 2, 2048)   0           ['add_35[0][0]']                 
                                                                                                  
 average_pooling2d_1 (AveragePo  (None, 1, 1, 2048)  0           ['activation_109[0][0]']         
 oling2D)                                                                                         
                                                                                                  
 flatten_1 (Flatten)            (None, 2048)         0           ['average_pooling2d_1[0][0]']    
          

In [ ]:
preds = pre_trained_model.evaluate(X_test, Y_test)

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
imshow(img)
prediction = pre_trained_model.predict(x)
print("Class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ", prediction)
print("Class:", np.argmax(prediction))